## Assignment 8

### Old code used in assignment 2

In [1]:
import numpy as np
import time

In [2]:
def swap(t,AB,n):
    for i in range(n):
        if AB[i][t]!=0:
            AB[t], AB[i] = AB[i], AB[t]
            break

def guassian(A,B):
    n=len(A)
    AB = []
    for i in range(n):
        AB.append(A[i] + [B[i]])
    for t in range(n):
        if AB[t][t] == 0:swap(t,AB,n)
        norm = AB[t][t]
        for i in range(t, n+1):
            AB[t][i]=AB[t][i]/norm
        for i in range(n):
            if i == t:continue
            norm = AB[i][t]
            for j in range(t, n+1):
                AB[i][j]=AB[i][j]- norm * AB[t][j]
    for i in range(n):
        if AB[i][i]==0 and AB[i][n]==0:
            return "The given system of linear equations has infinite solutions"
        elif AB[i][i]==0 and AB[i][n]!=0:
            return "The given system of linear equations has no solution"
    ans = [AB[i][n] for i in range(n)]
    return ans

In [3]:
import random
A=[]
for i in range(100):
    A.append([random.randint(1, 20) for i in range(100)])
B=[random.randint(1, 200) for i in range(100)]

#x=np.linalg.solve(A,B)
#print(x)
#%timeit np.linalg.solve(A,B)
#print(guassian(A,B))
%timeit guassian(A,B)

47.4 ms ± 1.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:

%reload_ext Cython


In [6]:
%%cython --annotate

import cython
import time
# Import the necessary Cython libraries
import numpy as np
import time
cimport numpy as np
from libc.stdlib cimport malloc, free

def swap(int t, np.ndarray[double, ndim=2] AB, int n):
    for i in range(n):
        if AB[i, t] != 0:
            AB[t, :], AB[i, :] = AB[i, :], AB[t, :]
            break

def guassian(np.ndarray[np.float64_t, ndim=2] A, np.ndarray[np.float64_t, ndim=2] B):
    start_time=time.time()
    cdef int n = len(A)
    cdef np.ndarray[double, ndim=2] AB = np.column_stack((A, B))
    cdef int t, i, j
    cdef double norm
    for t in range(n):
        if AB[t, t] == 0:
            swap(t, AB, n)
        norm = AB[t, t]
        for i in range(t, n + 1):
            AB[t, i] /= norm
        for i in range(n):
            if i == t:
                continue
            norm = AB[i, t]
            for j in range(t, n + 1):
                AB[i, j] -= norm * AB[t, j]
    for i in range(n):
        if AB[i, i] == 0 and AB[i, n] == 0:
            return None
        elif AB[i, i] == 0 and AB[i, n] != 0:
            return float('inf')
    ans = [AB[i, n] for i in range(n)]
    #print (ans)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time taken: {elapsed_time:.6f} seconds")

    return ans

def get_input2():

    X = np.random.rand(100,100)
    Y = np.random.rand(100,1)
    rows = 100
    columns =100
    #print(X)
    #print(Y)
    guassian(X, Y)
    return X, Y, rows, columns

get_input2()

Time taken: 0.000874 seconds


### Explanation

In this code we use cython to optimise the above old code 

Cython is a superset of Python that allows for the creation of C extensions for Python code. By adding static typing and other C language constructs to Python code, Cython can generate highly optimized C code that can be compiled into a Python extension module.<br>
Cython allows us to declare variable types using cdef, which allows the code to be compiled to machine code, making it faster than the equivalent Python code. In this implementation, we have declared all the variables that can have a fixed type (such as integers, doubles, and arrays) with the cdef keyword, which results in faster code execution.<br>
The code uses NumPy's column_stack function to concatenate the matrices A and B along the second axis, which is faster than using Python's zip function<br>
The code uses NumPy array indexing to access elements of the matrices, which is faster than Python's built-in list indexing

The %%cython --annotate magic command at the top of the code tells Jupyter to use Cython to compile the code and generate an annotated HTML file that shows how Cython has translated the Python code to C code.

In both cases I have given a 100 * 100 matrix and printed the time taken in these cases

As expected the cython code produces the required output in less time compared to the regular python code